In [1]:
import sgml, sgutil, sgpp, dproc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import root_mean_squared_error

In [2]:
from proc_v2 import p
from ml_v1 import X_cat, X_num, X_all, target, kf, ss, config
from ml_v1 import xgb_adapter, lgb_adapter, cb_adapter, lr_adapter, nn_adapter

2025-05-20 05:49:27.682966: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747720167.694590    2242 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747720167.698149    2242 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747720167.707117    2242 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747720167.707131    2242 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747720167.707133    2242 computation_placer.cc:177] computation placer alr

In [3]:
sc = sgutil.SGCache('img', 'result', 'model')
df_train = p.fit_transform(['data/train.csv']).assign(
    Calories_Log = lambda x: np.log(x['Calories'] + 1)
)
df_test = p.transform(['data/test.csv'])

# LGB 1

- Duration 대신에 Duration_log를 사용하고 나머지 변수를 넣습니다.

In [4]:
hparams = {
    'model_params': {'n_estimators': 2000, 'colsample_bytree': 0.9, 'num_leaves': 63, 'learning_rate': 0.03},
    'X_num': ['Sex', 'Age', 'Height', 'Weight', 'Duration_log', 'Heart_Rate', 'Body_Temp'], 
    #'validation_fraction': 0.1, 
}
result = sc.cv_result('lgb1', df_train, kf, hparams, config, lgb_adapter, rerun = 0)
np.mean(result['valid_scores']), result['valid_scores']

Fold:   0%|          | 0/4 [00:00<?, ?it/s]

(np.float64(0.06024029483999867),
 [np.float64(0.06015798639261961),
  np.float64(0.06100800453393292),
  np.float64(0.05898879823224211),
  np.float64(0.06080639020120003)])

# LGB 2

In [6]:
hparams = {
    'model_params': {'n_estimators': 2500, 'colsample_bytree': 0.9, 'num_leaves': 7, 'learning_rate': 0.07},
    'X_num': ['Sex', 'Age', 'Height', 'Weight', 'Duration_log', 'Heart_Rate', 'Body_Temp'], 
    #'validation_fraction': 0.1, 
}
result = sc.cv_result('lgb2', df_train, kf, hparams, config, lgb_adapter, rerun = 0)
np.mean(result['valid_scores']), result['valid_scores']

Fold:   0%|          | 0/4 [00:00<?, ?it/s]

(0.06022144346374213,
 [0.06017151518862502,
  0.06110662967724778,
  0.05892715299410521,
  0.06068047599499048])

In [7]:
root_mean_squared_error(
    df_train[target],
    sc.read_prds(['lgb2', 'lgb1'], index = df_train.index).dot([0.5, 0.5])
)

0.05983695633620083

# XGB1

In [8]:
hparams = {
    'model_params': {'n_estimators': 2000, 'colsample_bytree': 0.85, 'learning_rate': 0.05, 'max_depth': 4},
    'X_num': ['Sex', 'Age', 'Height', 'Weight', 'Duration_log', 'Heart_Rate', 'Body_Temp',
              'Heart_Rate_div_Weight_sqrt'], 
    #'validation_fraction': 0.1, 
}
result = sc.cv_result('xgb1', df_train, kf, hparams, config, xgb_adapter, rerun = 0, use_gpu = True)
np.mean(result['valid_scores']), result['valid_scores']

Fold:   0%|          | 0/4 [00:00<?, ?it/s]

/home/sun9sun9/python312/lib/python3.12/site-packages/xgboost/core.py:729: UserWarning: [21:33:40] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  return func(**kwargs)


(0.06027572602033615,
 [0.05999422445893288,
  0.061192434281110764,
  0.058978915214538574,
  0.06093733012676239])

# XGB2

In [9]:
hparams = {
    'model_params': {'n_estimators': 5000, 'colsample_bytree': 0.85, 'learning_rate': 0.03, 'max_depth': 3},
    'X_num': ['Sex', 'Age', 'Height', 'Weight', 'Duration_log', 'Heart_Rate', 'Body_Temp',
              'Heart_Rate_div_Weight_sqrt'], 
    #'validation_fraction': 0.1, 
}
result = sc.cv_result('xgb2', df_train, kf, hparams, config, xgb_adapter, rerun = 1, use_gpu = True)
np.mean(result['valid_scores']), result['valid_scores']

Fold:   0%|          | 0/4 [00:00<?, ?it/s]

(0.060211884789168835,
 [0.05990520119667053,
  0.06109195202589035,
  0.0589660219848156,
  0.06088436394929886])

# CB1

In [10]:
hparams = {
    'model_params': {'max_depth': 5, 'n_estimators': 2000, 'learning_rate': 0.1}, 
    'X_num': ['Sex', 'Age', 'Height', 'Weight', 'Duration_log', 'Heart_Rate', 'Body_Temp'], 
}

result = sc.cv_result('cb1', df_train, kf, hparams, config, cb_adapter, rerun = 0)
np.mean(result['valid_scores']), result['valid_scores']

Fold:   0%|          | 0/4 [00:00<?, ?it/s]

(0.05968437186609724,
 [0.05929705865156929,
  0.06071020397792236,
  0.058418971867058654,
  0.06031125296783865])

# CB2

In [11]:
hparams = {
    'model_params': {'max_depth': 6, 'n_estimators': 2000, 'learning_rate': 0.1}, 
    'X_num': ['Sex', 'Age', 'Height', 'Weight', 'Duration_log', 'Heart_Rate', 'Body_Temp'], 
}

result = sc.cv_result('cb2', df_train, kf, hparams, config, cb_adapter, rerun = 0)
np.mean(result['valid_scores']), result['valid_scores']

Fold:   0%|          | 0/4 [00:00<?, ?it/s]

(0.05972289513623428,
 [0.05955441198810907,
  0.06049828048324206,
  0.058326367256739016,
  0.06051252081684697])

# CB3

In [12]:
hparams = {
    'model_params': {'max_depth': 7, 'n_estimators': 2000, 'learning_rate': 0.07, 'colsample_bylevel': 0.8}, 
    'X_num': ['Sex', 'Age', 'Height', 'Weight', 'Duration_log', 'Heart_Rate', 'Body_Temp'], 
}

result = sc.cv_result('cb3', df_train, kf, hparams, config, cb_adapter, use_gpu = 0, rerun = 0)
np.mean(result['valid_scores']), result['valid_scores']

Fold:   0%|          | 0/4 [00:00<?, ?it/s]

(0.05958652534082724,
 [0.05942618972933533,
  0.06046098674510978,
  0.05827122680710294,
  0.060187698081760915])

# CB4

In [13]:
hparams = {
    'model_params': {'max_depth': 8, 'n_estimators': 4000, 'learning_rate': 0.02}, 
    'X_num': ['Sex', 'Age', 'Height', 'Weight', 'Duration_log', 'Heart_Rate', 'Body_Temp'], 
}

result = sc.cv_result('cb4', df_train, kf, hparams, config, cb_adapter, use_gpu = 1, rerun = 0)
np.mean(result['valid_scores']), result['valid_scores']

Fold:   0%|          | 0/4 [00:00<?, ?it/s]

(0.05964453014380632,
 [0.059431297182173874,
  0.0604458558048441,
  0.058415995996832025,
  0.06028497159137528])

# NN1

In [14]:
nn_params = {
    'config':  [
        {'unit': 64, 'activation': 'swish', 'batch_norm': False},
        {'unit': 128, 'activation': 'swish', 'batch_norm': False},
        {'unit': 128, 'activation': 'swish', 'batch_norm': False},
        {'unit': 64, 'activation': 'swish', 'batch_norm': False},
    ]
}
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * np.cos((epoch - 9) / 25 * np.pi)
hparams = {
    'model_params': {
        'model_params': nn_params,
        'epochs': 30,
        'optimizer': ('Adam', {'learning_rate': 0.0001}),
        'batch_size': 128, 'shuffle_size': 204800,
        'early_stopping': None, 'reduce_lr_on_plateau': None, 'lr_scheduler': {'schedule': scheduler}
    }, 'X_std': ['Age', 'Height', 'Weight', 'Duration', 'Duration_log', 'Heart_Rate', 'Heart_Rate_sqrt_d', 'Body_Temp'], 'X_num': ['Sex']
}

result = sc.cv_result('nn1', df_train, kf, hparams, config, nn_adapter, rerun = 1)

Fold:   0%|          | 0/4 [00:00<?, ?it/s]

Epoch:   0%|          | 0/30 [00:00<?, ?it/s]

Step:   0%|          | 0/4395 [00:00<?, ?it/s]

Epoch:   0%|          | 0/30 [00:00<?, ?it/s]

Step:   0%|          | 0/4395 [00:00<?, ?it/s]

Epoch:   0%|          | 0/30 [00:00<?, ?it/s]

Step:   0%|          | 0/4395 [00:00<?, ?it/s]

Epoch:   0%|          | 0/30 [00:00<?, ?it/s]

Step:   0%|          | 0/4395 [00:00<?, ?it/s]

In [15]:
np.mean(result['valid_scores']), result['valid_scores']

(0.05964453014380632,
 [0.059431297182173874,
  0.0604458558048441,
  0.058415995996832025,
  0.06028497159137528])

# NN2

In [11]:
nn_params = {
    'config':  [
        {'unit': 128, 'activation': 'swish', 'batch_norm': False},
        {'unit': 256, 'activation': 'swish', 'batch_norm': False},
        {'unit': 256, 'activation': 'swish', 'batch_norm': False},
        {'unit': 128, 'activation': 'swish', 'batch_norm': False},
    ]
}

hparams = {
    'model_params': {
        'model_params': nn_params,
        'epochs': 30,
        'optimizer': ('Adam', {'learning_rate': 0.0001}),
        'batch_size': 128, 'shuffle_size': 204800,
        'early_stopping': None, 'reduce_lr_on_plateau': None, 'lr_scheduler': {'schedule': scheduler}
    }, 'X_std': ['Age', 'Height', 'Weight', 'Duration', 'Duration_log', 'Heart_Rate', 'Heart_Rate_sqrt_d', 'Body_Temp'], 'X_num': ['Sex']
}

result = sc.cv_result('nn2', df_train, kf, hparams, config, nn_adapter, rerun = 0)
np.mean(result['valid_scores']), result['valid_scores']

(0.0599099975079298,
 [0.05969740077853203,
  0.06047586351633072,
  0.05873413011431694,
  0.06073259562253952])

# NN3

In [16]:
nn_params = {
    'config':  [
        {'unit': 64, 'activation': 'swish', 'batch_norm': False},
        {'unit': 128, 'activation': 'swish', 'batch_norm': False},
        {'unit': 256, 'activation': 'swish', 'batch_norm': False},
        {'unit': 64, 'activation': 'swish', 'batch_norm': False},
    ]
}

hparams = {
    'model_params': {
        'model_params': nn_params,
        'epochs': 30,
        'optimizer': ('Adam', {'learning_rate': 0.0001}),
        'batch_size': 128, 'shuffle_size': 204800,
        'early_stopping': None, 'reduce_lr_on_plateau': None, 'lr_scheduler': {'schedule': scheduler}
    }, 'X_std': ['Age', 'Height', 'Weight', 'Duration', 'Duration_log', 'Heart_Rate', 'Heart_Rate_sqrt_d', 'Body_Temp'], 'X_num': ['Sex']
}

result = sc.cv_result('nn3', df_train, kf, hparams, config, nn_adapter, rerun = 0)
np.mean(result['valid_scores']), result['valid_scores']

Fold:   0%|          | 0/4 [00:00<?, ?it/s]

Epoch:   0%|          | 0/30 [00:00<?, ?it/s]

Step:   0%|          | 0/4395 [00:00<?, ?it/s]

Epoch:   0%|          | 0/30 [00:00<?, ?it/s]

Step:   0%|          | 0/4395 [00:00<?, ?it/s]

Epoch:   0%|          | 0/30 [00:00<?, ?it/s]

Step:   0%|          | 0/4395 [00:00<?, ?it/s]

Epoch:   0%|          | 0/30 [00:00<?, ?it/s]

Step:   0%|          | 0/4395 [00:00<?, ?it/s]

(0.059880093671381474,
 [0.05979902297258377,
  0.06053208187222481,
  0.0588604137301445,
  0.060328856110572815])

# Ensemble

In [24]:
df_stk = sc.read_prds(
    ['lgb2', 'lgb1', 'xgb1', 'xgb2', 'cb1', 'cb2', 'cb3', 'cb4', 'nn1', 'nn2', 'nn3'], index = df_train.index
).assign(
    Calories_Log = df_train[target]
)

## Voting

In [19]:
root_mean_squared_error(
    df_train[target], df_stk.iloc[:, :-1].mean(axis = 1)
)

0.05940315042664131

## Stacking

In [25]:
X_stk = ['lgb2', 'lgb1', 'xgb1', 'xgb2', 'cb1', 'cb2', 'cb3', 'cb4', 'nn1', 'nn2']
hparams = {
    'X_num': X_stk
}
result = sgml.cv(df_stk, kf, hparams, config, lr_adapter)
np.mean(result['valid_scores'])

Fold:   0%|          | 0/4 [00:00<?, ?it/s]

0.0593831595602206

In [28]:
for i in X_stk:
    sc.train_cv(i, df_train, config)

Epoch:   0%|          | 0/30 [00:00<?, ?it/s]

Step:   0%|          | 0/5860 [00:00<?, ?it/s]

Epoch:   0%|          | 0/30 [00:00<?, ?it/s]

Step:   0%|          | 0/5860 [00:00<?, ?it/s]

In [35]:
from sklearn.linear_model import LinearRegression
reg_meta_lr = LinearRegression()
reg_meta_lr.fit(df_stk[X_stk], df_train[target])

LinearRegression()

In [36]:
df_stk_test = pd.concat([
    sc.get_predictor_cv(i, config)(df_test).rename(i) for i in X_stk
], axis = 1)
df_stk_test.head()

,lgb2,lgb1,xgb1,xgb2,cb1,cb2,cb3,cb4,nn1,nn2
id,,,,,,,,,,
750000,3.332904,3.330881,3.331915,3.341200,3.333360,3.339220,3.337554,3.337476,3.345670,3.343016
750001,4.701861,4.698709,4.702695,4.700873,4.692424,4.691211,4.689848,4.688161,4.694196,4.691476
750002,4.456351,4.469250,4.475701,4.468938,4.473388,4.476912,4.481037,4.479083,4.482096,4.475423
750003,4.835725,4.828052,4.834696,4.844317,4.841058,4.841173,4.838728,4.839390,4.839857,4.841015
750004,4.335238,4.355100,4.339209,4.332186,4.345081,4.345119,4.341792,4.340342,4.346127,4.347120


In [40]:
pd.Series(
    reg_meta_lr.predict(df_stk_test), index = df_stk_test.index, name = 'Calories'
).pipe(lambda x: (np.exp(x) - 1).clip(lower = 1)).to_csv('result/submission1.csv')

In [41]:
!head result/submission1.csv

id,Calories
750000,27.177677490888385
750001,108.18015160403712
750002,86.97043885735042
750003,125.25525410877265
750004,76.17928716888093
750005,21.741995587381073
750006,48.75503859934514
750007,6.773944607784969
750008,10.072104732929219


In [42]:
# !kaggle competitions submit -c playground-series-s5e5 -f result/submission1.csv -m "Ensemble1"

100%|██████████████████████████████████████| 6.06M/6.06M [00:02<00:00, 2.17MB/s]
Successfully submitted to Predict Calorie Expenditure